# 2a

In [ ]:
def load_and_sort_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        n = int(lines[0].strip())
        points = [tuple(map(float, line.strip().split())) for line in lines[1:]]
    points.sort(key=lambda x: x[0])
    return points

In [91]:
def compute_prefix_sums(points):
    # Initialize prefix sums for x, y, x^2, xy, and y^2
    n = len(points)
    sum_x, sum_y, sum_x2, sum_xy, sum_y2 = [0] * (n + 1), [0] * (n + 1), [0] * (n + 1), [0] * (n + 1), [0] * (n + 1)
    
    # Calculate prefix sums for each component
    for i in range(1, n + 1):
        x, y = points[i - 1]
        sum_x[i] = sum_x[i - 1] + x
        sum_y[i] = sum_y[i - 1] + y
        sum_x2[i] = sum_x2[i - 1] + x**2
        sum_xy[i] = sum_xy[i - 1] + x*y
        sum_y2[i] = sum_y2[i - 1] + y**2
    return sum_x, sum_y, sum_x2, sum_xy, sum_y2

def compute_least_squares_error(i, j, sum_x, sum_y, sum_x2, sum_xy, sum_y2):
    # Compute the least squares error and regression coefficients for a segment of points from i to j
    N = j - i + 1
    # Retrieve the required sums for the segment.
    sx, sy, sxx, sxy, syy = sum_x[j] - sum_x[i-1], sum_y[j] - sum_y[i-1], sum_x2[j] - sum_x2[i-1], sum_xy[j] - sum_xy[i-1], sum_y2[j] - sum_y2[i-1]
    denom = N * sxx - sx ** 2
    if denom == 0:  # Check for division by zero
        return float('inf'), 0, 0  
    b = (N * sxy - sx * sy) / denom  # Slope
    a = (sy - b * sx) / N  # Intercept
    error = syy - 2 * a * sy - 2 * b * sxy + a**2 * N + 2 * a * b * sx + b**2 * sxx
    return error, a, b

def segmented_least_squares_with_coefficients(points, l=3):
    # Determine the optimal way to partition points into segments with linear regression
    n = len(points)
    # Compute prefix sums needed for error calculations.
    sum_x, sum_y, sum_x2, sum_xy, sum_y2 = compute_prefix_sums(points)
    
    # Initialize DP table for storing minimum error and backtracking information
    dp = [[float('inf')] * (l + 1) for _ in range(n + 1)]
    dp[0][0] = 0  # Base case: no error with 0 points.
    backtrack = [[None] * (l + 1) for _ in range(n + 1)]
    
    # Fill DP table with minimum errors and update backtracking information
    # Nested loops contribute to O(n^2*l) time complexity, as we explore all subproblems
    for j in range(1, n + 1):
        for k in range(1, min(l + 1, j + 1)):
            for i in range(max(1, 2 * k - 1), j + 1):
                error, _, _ = compute_least_squares_error(i, j, sum_x, sum_y, sum_x2, sum_xy, sum_y2)
                if dp[i - 1][k - 1] + error < dp[j][k]:
                    dp[j][k] = dp[i - 1][k - 1] + error
                    backtrack[j][k] = i
    
    # Backtrack to construct segments and calculate their regression coefficients
    segments = []
    current_l = l
    current_j = n
    while current_l > 0 and current_j > 0:
        start = backtrack[current_j][current_l]
        _, a, b = compute_least_squares_error(start, current_j, sum_x, sum_y, sum_x2, sum_xy, sum_y2)
        x_start, _ = points[start-1] if start > 0 else points[0]
        x_end, _ = points[current_j-1]
        segments.append((x_start, x_end, a, b))
        current_j = start - 1
        current_l -= 1
    
    segments.reverse()
    return segments

# Calculate the segments and their coefficients
segments_with_coefficients = segmented_least_squares_with_coefficients(points, l=3)

# Output the results
for segment in segments_with_coefficients:
    x_start, x_end, a, b = segment
    print(f"Line segment from x = {x_start} to x = {x_end} with equation: y = {a:.4f} + {b:.4f}x")


Line segment from x = 0.599 to x = 168.6967 with equation: y = 41.4170 + 2.6199x
Line segment from x = 170.4067 to x = 341.5237 with equation: y = 499.5263 + -0.0828x
Line segment from x = 343.1398 to x = 498.5574 with equation: y = 1428.2028 + -2.7978x


# 2b

In [92]:
def compute_total_cost(i, j, error, C):
    """
    Computes the total cost for a segment from i to j, considering the error and the fixed cost per line, C.
    Time Complexity: O(1), as it performs basic arithmetic operations.
    """
    return error + C

def bottom_up_dp_with_coefficients(points, C=10000):
    # Computes segments with linear regression coefficients that minimize the total cost of fitting.
    # C is the cost per line segment.
    n = len(points)  # Length of the points array.

    # Calculate prefix sums for efficient computation of linear regression errors.
    sum_x, sum_y, sum_x2, sum_xy, sum_y2 = compute_prefix_sums(points)

    # DP table initialization.
    dp = [float('inf')] * (n + 1)  # Stores the minimum cost to fit up to the j-th point.
    dp[0] = 0  
    # Backtrack table to reconstruct the solution.
    backtrack = [0] * (n + 1)  # Tracks the start index of the segment for the optimal solution.

    # Iterate over all end points of segments
    # results in O(n^2) time complexity as it explores all possible segments.
    for j in range(1, n + 1):
        # Iterate over all possible start points for a segment ending at j
        for i in range(1, j + 1):
            # Calculate the error and the total cost for segment [i, j].
            error, a, b = compute_least_squares_error(i, j, sum_x, sum_y, sum_x2, sum_xy, sum_y2)
            total_cost = dp[i - 1] + compute_total_cost(i, j, error, C)
            # Update DP table if a lower cost is found.
            if total_cost < dp[j]:
                dp[j] = total_cost
                backtrack[j] = i
    
    # Reconstruct the solution from the DP and backtrack tables
    segments = []
    current = n
    while current > 0:
        start = backtrack[current]
        _, a, b = compute_least_squares_error(start, current, sum_x, sum_y, sum_x2, sum_xy, sum_y2)
        x_start, _ = points[start-1] if start > 0 else points[0]
        x_end, _ = points[current-1]
        segments.append((x_start, x_end, a, b))
        current = start - 1
    
    segments.reverse()  

    return segments


segments_with_coefficients = bottom_up_dp_with_coefficients(points, C)


for segment in segments_with_coefficients:
    x_start, x_end, a, b = segment
    print(f"Line segment from x = {x_start} to x = {x_end} with equation: y = {a:.4f} + {b:.4f}x")




Line segment from x = 0.599 to x = 86.9667 with equation: y = 11.4751 + 3.2699x
Line segment from x = 88.7357 to x = 163.5283 with equation: y = 125.4943 + 1.9756x
Line segment from x = 164.7958 to x = 250.3958 with equation: y = 334.1240 + 0.7051x
Line segment from x = 252.0358 to x = 341.5237 with equation: y = 704.0777 + -0.7672x
Line segment from x = 343.1398 to x = 421.9327 with equation: y = 1182.4614 + -2.1598x
Line segment from x = 422.7515 to x = 498.5574 with equation: y = 1693.9691 + -3.3707x


# 3a

In [79]:
def can_distribute_trash_bags(n, w, M):
    # O(n*M^3) - The DP table is 4D with dimensions n+1 by M+1 by M+1 by M+1.
    # This is because for each trash bag, we explore all possible distributions of weights across the 3 containers.
    DP = [[[[False for _ in range(M+1)] for _ in range(M+1)] for _ in range(M+1)] for _ in range(n+1)]
    
    # O(1) - Base case initialization
    DP[0][0][0][0] = True
    
    # O(n) - We iterate through each trash bag
    for i in range(1, n+1):
        # O(M^3) - For each trash bag, we explore all possible distributions of its weight across the 3 containers.
        # The nested loops go through all combinations of weights that can be in the 3 containers.
        for j in range(M+1):
            for k in range(M+1):
                for l in range(M+1):
                    # Constant time operations within the innermost loop
                    if not DP[i-1][j][k][l]:
                        continue
                    
                    # O(1) - Updating the DP table for each possible distribution
                    if j + w[i-1] <= M:
                        DP[i][j + w[i-1]][k][l] = True
                    if k + w[i-1] <= M:
                        DP[i][j][k + w[i-1]][l] = True
                    if l + w[i-1] <= M:
                        DP[i][j][k][l + w[i-1]] = True
    
    # O(M^3) - Final check over all possible weight distributions in 3 containers.
    # We look for any valid distribution that fits all bags.
    for j in range(M+1):
        for k in range(M+1):
            for l in range(M+1):
                if DP[n][j][k][l]:
                    return True
    return False

n = int(input("Enter the number of trash bags: "))
w = [int(x) for x in input("Enter the weights of the trash bags, separated by spaces: ").split()]
M = int(input("Enter the maximum weight capacity of the containers: "))

can_distribute_trash_bags(n, w, M)


 5
 2 2 3 4 5
 5


False

# 3b

In [82]:
def can_fit_trash_bags_optimized(n, w, M):
    # O(M) - Initialize the DP array with M+1 elements
    DP = [False] * (M + 1)
    DP[0] = True  # Base case: Achieving 0 weight is always possible

    # O(nM) - Main loop: For each trash bag (n), iterate through potential weights up to M
    for weight in w:
        # O(M) - Update the DP array in reverse for each weight
        for j in range(M, weight - 1, -1):
            DP[j] = DP[j] or DP[j - weight]
    
    # O(M) - Find the maximum achievable weight that does not exceed M by iterating through DP array
    for j in range(M, -1, -1):
        if DP[j]:
            # Constant time check - O(1)
            if sum(w) - j <= M:
                return True
    return False

n = int(input("Enter the number of trash bags: "))
w = [int(x) for x in input("Enter the weights of the trash bags, separated by spaces: ").split()]
M = int(input("Enter the maximum weight capacity of the containers: "))

can_fit_trash_bags_optimized(n, w, M) 



Enter the number of trash bags:  5
Enter the weights of the trash bags, separated by spaces:  2 2 3 4 5
Enter the maximum weight capacity of the containers:  5


False

# 4

In [86]:
A = [int(x) for x in input("Enter the elements of array A separated by space: ").split()]
X = int(input())

def find_max_subarray_dp_optimized(A, X):
    n = len(A)  # O(1), getting the length of the array
    
    # Calculate prefix sums for constant-time sum queries
    prefix_sums = [0]  # O(1), initializing the prefix sums list
    for num in A:
        prefix_sums.append(prefix_sums[-1] + num)  # O(n), as appending to a list is O(1) but done n times
    
    # Initialize variables for tracking the maximum sum and positions
    max_sum = float('-inf')  # O(1), initializing variables
    L = R = 0  # O(1), initializing variables

    # Dynamic Programming with Rolling Sum
    # We iterate once through the array, using the prefix sums to quickly calculate the sum of subarrays of length >= X and updating the maximum sum and positions accordingly
    for i in range(X, n + 1):  # O(n-X+1) which simplifies to O(n), iterating from X to n
        current_sum = prefix_sums[i] - prefix_sums[i - X]  # O(1), calculating sum using prefix sums
        if current_sum > max_sum:
            max_sum = current_sum  # O(1), updating max_sum
            L, R = i - X, i - 1  # O(1), updating L and R
    
    # We return L and R as the starting and ending indices of the subarray

    return L, R, max_sum

L, R, max_sum = find_max_subarray_dp_optimized(A, X)

print(f'The max sub array sum is {max_sum} with L = {L} and R = {R}.')



Enter the elements of array A separated by space:  1 -2 -3 10 4 -7 2 -5
 5


The max sub array sum is 10 with L = 0 and R = 4.


# 5

In [83]:
def count_shuffles(A, B, C):
    m, n = len(A), len(B)
    if len(C) != m + n:  # Quick check
        return 0
    
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    dp[0][0] = 1  # Base case: one way to form an empty string
    
    # Fill the DP table
    for i in range(m + 1):
        for j in range(n + 1):
            if i > 0 and A[i - 1] == C[i + j - 1]:
                dp[i][j] += dp[i - 1][j]
            if j > 0 and B[j - 1] == C[i + j - 1]:
                dp[i][j] += dp[i][j - 1]
                
    return dp[m][n]

A = input("Enter string A: ")
B = input("Enter string B: ")
C = input("Enter string C: ")

# Calculate and print the number of shuffles
num_shuffles = count_shuffles(A, B, C)
print(f"The number of ways C can be a shuffle of A and B is: {num_shuffles}")


Enter string A:  BANANA
Enter string B:  ANANAS
Enter string C:  BANANAANANAS


The number of ways C can be a shuffle of A and B is: 12
